
# New Clustering of Breast Cancer Subtypes Using Gene Expression Data and the BIGCLAM Method
_(Training BIGCLAM(GPU) model on breast cancer gene expression data)_


## 📦 Import Necessary Libraries

In [ ]:
# Data manipulation and utilities
import os
import json
import pickle
import joblib
import numpy as np
import pandas as pd

# Graphs and visualization
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt

# Machine learning and preprocessing
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import (
    confusion_matrix,
    roc_curve,
    roc_auc_score,
    accuracy_score,
    matthews_corrcoef,
)

## 🛠️ Data Prepration

### 📂 Load the dataset

In [ ]:
# PAM50 column from clinical dataset has been added to genomic dataset as target
df = pd.read_csv('/content/drive/MyDrive/gene_expression_data_target_added.csv')

#making columns as features
data = df.T

#removing PAM50 column from dataset
X = data.iloc[1:,:]

### Feature Reduction(Variance)

In [ ]:
indices_names = data.iloc[0, :-1]  # Get the names of the indices from the first row

# Initialize the VarianceThreshold object(threshold=13 founded through trial and errors)
var_threshold = VarianceThreshold(threshold=13)

# Fit the VarianceThreshold to data
var_threshold.fit(X)

# Get the indices of the features to keep
selected_features = var_threshold.get_support(indices=True)

# Extract the names of the selected features using the first row of 'data'
selected_feature_names = indices_names.iloc[selected_features].values

# Create a DataFrame to display the selected feature indices and their corresponding names
df_selected_features = pd.DataFrame({
    "Selected Feature Index": selected_features,
    "Feature Name": selected_feature_names
})

# Export the DataFrame to an Excel file
#df_selected_features.to_excel("selected_features_with_names.xlsx", index=False)

# Print a confirmation message
print("Selected features have been exported to 'selected_features_with_names.xlsx'")

# Subset the original data with the selected features
X_reduced = X.iloc[:, selected_features]
# X_reduced = pd.concat([X_reduced,data.iloc[1: , -1]],axis = 1)
# Check the shape of the reduced data
print("Original data shape:", X.shape)
print("Reduced data shape:", X_reduced.shape)
print(X_reduced)

##  Graph Making

In [ ]:
def create_adjacency_and_assignments(df, threshold=0.4):
    N = df.shape[0]
    num_features = df.shape[1] - 1  # last column is community

    from sklearn.metrics.pairwise import cosine_similarity

    features = df.iloc[:, :num_features].values
    similarity_matrix = cosine_similarity(features)

    A = (similarity_matrix > threshold).astype(int) - np.eye(N).astype(int)

    community_assignments = df.iloc[:, -1].apply(lambda x: {int(x)}).tolist()

    return A, community_assignments

def gen_json(A, p2c, F_argmax=None):
    N = A.shape[0]
    data = {'nodes':[], 'links':[]}

    for i in range(N):
        grp = ''.join(map(str, sorted(p2c[i])))
        node = {'id': str(i), 'group': str(grp)}
        if F_argmax is not None:
            node.update({'assigned': str(F_argmax[i])})
        data['nodes'].append(node)
        friends = np.where(A[i])
        for friend in friends[0]:
            inter = 2 - len(p2c[i].intersection(p2c[friend]))
            data['links'].append({'source': str(i), 'target': str(friend), 'value': str(inter * 5 + 1), 'distance': str(inter * 15 + 1)})
    return data

# Load dataset
df = X_reduced

# Create adjacency matrix and community assignments
A, p2c = create_adjacency_and_assignments(df)

# Generate JSON data
data = gen_json(A, p2c)

# Save the data
with open('data.json', 'w') as f:
    json.dump(data, f, indent=4)

# Define the path in your Google Drive where you want to save the files
save_path = '/content/drive/MyDrive/BIGCLAM_data/'

# Create the directory if it doesn't exist
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Save the adjacency matrix and community assignments
np.save(os.path.join(save_path, 'adj.npy'), A + A.T)
pickle.dump(p2c, open(os.path.join(save_path, "p2c.pkl"), "wb"))

print(f"Files saved to Google Drive at: {save_path}")

Files saved to Google Drive at: /content/drive/MyDrive/TCGA/


## 🚀 Initialize the BIGCLAM Model on GPU

In [ ]:
# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def sigm(x):
    return torch.div(torch.exp(-1. * x), 1. - torch.exp(-1. * x))

def log_likelihood(F, A):
    A_soft = F @ F.T  # Matrix multiplication in PyTorch
    FIRST_PART = A * torch.log(1. - torch.exp(-1. * A_soft))
    sum_edges = torch.sum(FIRST_PART)
    SECOND_PART = (1 - A) * A_soft
    sum_nedges = torch.sum(SECOND_PART)
    log_likeli = sum_edges - sum_nedges
    return log_likeli

def gradient(F, A, i):
    N, C = F.shape
    neighbours = torch.where(A[i])[0]
    nneighbours = torch.where(1 - A[i])[0]
    sum_neigh = torch.zeros((C,), device=device)
    for nb in neighbours:
        dotproduct = F[nb].dot(F[i])
        sum_neigh += F[nb] * sigm(dotproduct)
    sum_nneigh = torch.zeros((C,), device=device)
    for nnb in nneighbours:
        sum_nneigh += F[nnb]
    grad = sum_neigh - sum_nneigh
    return grad

def train(A, max_communities=10, iterations=100):
    N = A.shape[0]
    best_F = None
    best_num_communities = 1
    best_aic = float('inf')
    for num_communities in range(1, max_communities + 1):
        F = torch.rand((N, num_communities), device=device, requires_grad=True)
        optimizer = torch.optim.Adam([F], lr=0.08)  # Use an optimizer for better convergence
        for n in range(iterations):
            optimizer.zero_grad()
            ll = log_likelihood(F, A)
            loss = -ll  # Minimize negative log-likelihood
            loss.backward()
            optimizer.step()
            with torch.no_grad():
                F.data = torch.clamp(F.data, min=0.000000000001)  # Ensure F is nonnegative
        # Calculate AIC
        k = N * num_communities  # Number of parameters
        aic = -2 * ll.item() + 2 * k
        if aic < best_aic:
            best_aic = aic
            best_F = F.detach().cpu().numpy()  # Move back to CPU for compatibility
            best_num_communities = num_communities
        print(f'At step {num_communities}/{max_communities}, AIC: {aic}')
    print(f"Best number of communities: {best_num_communities}")
    return best_F

def gen_json(A, p2c, F_argmax=None):
    N = A.shape[0]
    data = {'nodes': [], 'links': []}
    for i in range(N):
        grp = ''.join(map(str, sorted(p2c[i])))
        node = {'id': str(i), 'group': str(grp)}
        if F_argmax is not None:
            node.update({'assigned': str(F_argmax[i])})
        data['nodes'].append(node)
        friends = torch.where(A[i])[0].cpu().numpy()  # Ensure A is a PyTorch tensor
        for friend in friends:
            inter = 2 - len(p2c[i].intersection(p2c[friend]))
            data['links'].append({'source': str(i), 'target': str(friend), 'value': str(inter * 5 + 1), 'distance': str(inter * 15 + 1)})
    return data

def visualize_bipartite(F, p2c):
    B = nx.Graph()
    N, C = F.shape
    person_nodes = ["Person {}".format(i) for i in range(N)]
    community_nodes = ["Community {}".format(i) for i in range(C)]

    B.add_nodes_from(person_nodes, bipartite=0)
    B.add_nodes_from(community_nodes, bipartite=1)

    for i, comm_prefs in enumerate(F):
        for c, membership in enumerate(comm_prefs):
            if membership > 0:
                B.add_edge(person_nodes[i], community_nodes[c])

    pos = nx.drawing.layout.bipartite_layout(B, person_nodes)
    plt.figure(figsize=(12, 8))
    nx.draw(B, pos, with_labels=True, node_color=['skyblue' if n in person_nodes else 'lightgreen' for n in B.nodes()], edge_color='gray')
    plt.title('Bipartite Graph of People and Communities')
    plt.show()

if __name__ == "__main__":
    # Define the path in your Google Drive where the files are saved
    load_path = '/content/drive/MyDrive/BIGCLAM_data/'

    # Load the adjacency matrix and community assignments
    adj = np.load(os.path.join(load_path, 'adj.npy'))
    p2c = pickle.load(open(os.path.join(load_path, 'p2c.pkl'), 'rb'))

    # Convert adjacency matrix to a PyTorch tensor and move to GPU
    adj = torch.tensor(adj, device=device)

    # Number of communities (adjust this based on data)
    num_communities = 4

    # Train the BigClAM model
    F = train(adj, num_communities)
    F_argmax = np.argmax(F, 1)

    # Generate the JSON data (pass adj as a tensor)
    data = gen_json(adj, p2c, F_argmax)

    # Save the generated JSON data
    with open('data.json', 'w') as f:
        json.dump(data, f, indent=4)

    # Optionally print the results
    for i, row in enumerate(F):
        print(f"Person {i}: Community Preferences - {row}")
        print(f"Person {i}: Assigned Communities - {p2c[i]}")

    # Visualize as a bipartite graph
    visualize_bipartite(F, p2c)

# Add the community labels as a new column to the dataframe
df = X_reduced
df['Community'] = F_argmax
# Save the updated dataframe
df.to_csv('/content/drive/MyDrive/reduced_data_BC_label.csv', index=False)

## counts of each class from new clustered dataset

In [ ]:
# load the clustered dataset as X
X = pd.read_csv('/content/drive/MyDrive/reduced_data_BC_label.csv')
# counts of each class
class_counts = X.iloc[:, -1].value_counts()
print(class_counts)

## Comparing PAM50 with new label per class

In [ ]:
# Load the datasets
data_with_community = pd.read_csv('/content/drive/MyDrive/reduced_data_BC_label.csv')

# Upploading dataset containg PAM50 column as target
data = pd.read_csv('/content/drive/MyDrive/gene_expression_data_target_added.csv')
data = data.T
data.iloc[:,-1] = data.iloc[:,-1].replace('pam50 subtype: Normal', 0)
data.iloc[:,-1] = data.iloc[:,-1].replace('pam50 subtype: LumA', 1)
data.iloc[:,-1] = data.iloc[:,-1].replace('pam50 subtype: LumB', 2)
data.iloc[:,-1] = data.iloc[:,-1].replace('pam50 subtype: Her2', 3)
data.iloc[:,-1] = data.iloc[:,-1].replace('pam50 subtype: Basal', 4)
#data
data = data[data.iloc[:, -1]!=0]
label_data = data.iloc[1:,-1]
label_data

# Assign the labels to the last column of data_with_community
data_with_community['label'] = label_data.values

# Ensure the columns are named correctly
if 'Community' not in data_with_community.columns:
    data_with_community.rename(columns={data_with_community.columns[-2]: 'Community'}, inplace=True)
if 'label' not in data_with_community.columns:
    data_with_community.rename(columns={data_with_community.columns[-1]: 'label'}, inplace=True)

# Count the number of each label in each community
label_counts_per_community = data_with_community.groupby('Community')['label'].value_counts().unstack().fillna(0).astype(int)

# Display the results
for community, labels in label_counts_per_community.iterrows():
    print(f"Community {community}:")
    for label, count in labels.items():
        print(f"  class {int(label)}: {count}")

# 🔍 Evaluate the model
(training MLP and SVM models using BIGCLAM clusterd gene expression data)

## 🛠️ Preprocess the Data

### Data Augmentation

In [ ]:
import numpy as np
import pandas as pd

# Separate features and labels
features = X.iloc[:, :-1].values
labels = X.iloc[:, -1].values

# Find the maximum number of samples among all classes
max_samples = max(np.sum(labels == label) for label in set(labels))

# Create a dictionary to store the indices of samples for each class
class_indices = {label: np.where(labels == label)[0] for label in set(labels)}

# Create an empty list to store augmented data
augmented_data = []

# Augment each class to reach the maximum number of samples
for label, indices in class_indices.items():
    # Calculate the number of augmentations needed for this class
    num_augmentations = max_samples - len(indices)

    # Randomly sample existing examples from this class
    sampled_indices = np.random.choice(indices, num_augmentations, replace=True)

    # Apply augmentations to each sampled example
    for index in sampled_indices:
        feature_row = features[index, :]

        # Add random noise to the gene expression values
        noise = np.random.normal(0, 0.1, feature_row.shape)
        augmented_feature_row = feature_row + noise

        augmented_data.append(np.concatenate([augmented_feature_row, [label]]))

# Convert the augmented data to a DataFrame
augmented_data = pd.DataFrame(augmented_data, columns=X.columns)

# Concatenate the original and augmented data
augmented_data = pd.concat([X, augmented_data], ignore_index=True)
class_counts = augmented_data.iloc[:, -1].value_counts()
print("Class counts in augmented_data:\n", class_counts)


### ✂️ Data Splitting and normalization

In [ ]:
import pandas as pd

# Assuming `augmented_data` is your dataset and the last column is the class label
# Example: augmented_data = pd.DataFrame(...)

# Initialize empty DataFrames for train, validation, and test sets
X_train = pd.DataFrame()
X_valid = pd.DataFrame()
X_test = pd.DataFrame()

# Iterate over each class
for class_label in augmented_data.iloc[:, -1].unique():
    # Filter data for the current class
    class_data = augmented_data[augmented_data.iloc[:, -1] == class_label]

    # Calculate the number of samples for each split
    total_samples = len(class_data)
    test_size = int(0.2 * total_samples)  # 10% for test
    valid_size = int(0.2 * total_samples)  # 10% for validation
    train_size = total_samples - test_size - valid_size  # 80% for train

    # Split the data
    test_data = class_data.iloc[:test_size]  # First 10% for test
    valid_data = class_data.iloc[test_size:test_size + valid_size]  # Next 10% for validation
    train_data = class_data.iloc[test_size + valid_size:]  # Remaining 80% for train

    # Append to the respective sets
    X_test = pd.concat([X_test, test_data])
    X_valid = pd.concat([X_valid, valid_data])
    X_train = pd.concat([X_train, train_data])

# Shuffle the datasets
X_test = X_test.sample(frac=1).reset_index(drop=True)
X_valid = X_valid.sample(frac=1).reset_index(drop=True)
X_train = X_train.sample(frac=1).reset_index(drop=True)

# Verify the splits
print(f'Test set size: {X_test.shape}')
print(f'Validation set size: {X_valid.shape}')
print(f'Train set size: {X_train.shape}')

# Define feature columns and target column for each set
y_train = X_train.iloc[:,-1]
y_valid = X_valid.iloc[:,-1]
y_test = X_test.iloc[:,-1]

X_train = X_train.iloc[:,:-1]
X_valid = X_valid.iloc[:,:-1]
X_test = X_test.iloc[:,:-1]

# Normalize the data
scaler = MinMaxScaler(feature_range=(-1, 1))
X_train = scaler.fit_transform(X_train)
X_valid = scaler.fit_transform(X_valid)
X_test = scaler.transform(X_test)
print(X_train)
print(X_valid)
print(X_test)

### Turning target into one-hot format

In [ ]:
# One-hot encode the categorical labels
encoder = OneHotEncoder(sparse_output=False)#, categories='auto')
y_train_onehot = encoder.fit_transform(y_train.values.reshape(-1, 1))
y_valid_onehot = encoder.transform(y_valid.values.reshape(-1, 1))
y_test_onehot = encoder.transform(y_test.values.reshape(-1, 1))
print(y_valid_onehot.shape)

## MLP model

In [ ]:
# Extract metrics from confusion matrix
def calculate_metrics(cm):
    TP = cm[1, 1]
    TN = cm[0, 0]
    FP = cm[0, 1]
    FN = cm[1, 0]

    sensitivity = TP / (TP + FN) if (TP + FN) != 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
    mcc = ((TP * TN) - (FP * FN)) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)) \
        if (TP + FP) * (TP + FN) * (TN + FP) * (TN + FN) != 0 else 0

    return TP, TN, FP, FN, sensitivity, specificity, mcc

# Define the improved MLP model
class MLP(nn.Module):
    def __init__(self, input_size, h1, h2, h3, output_size, dropout_rate=0.3):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, h1)
        self.bn1 = nn.BatchNorm1d(h1)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.leakyrelu = nn.LeakyReLU()

        self.fc2 = nn.Linear(h1, h2)
        self.bn2 = nn.BatchNorm1d(h2)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.fc3 = nn.Linear(h2, h3)
        self.bn3 = nn.BatchNorm1d(h3)
        self.dropout3 = nn.Dropout(dropout_rate)

        self.fc4 = nn.Linear(h3, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.leakyrelu(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.leakyrelu(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = self.leakyrelu(x)
        x = self.dropout3(x)

        x = self.fc4(x)
        x = self.softmax(x)
        return x

# Function to train and evaluate the model with early stopping
def train_and_evaluate(X_train, y_train_onehot, X_valid, y_valid_onehot, X_test, y_test_onehot,
                       num_epochs=200, lr=0.01, patience=10, weight_decay=1e-4, dropout_rate=0.3):
    input_size = len(X_train[0])  # Input size
    h1, h2, h3 = 80, 50, 20  # Reduced complexity
    output_size = len(y_train_onehot[0])  # Output size for one-hot encoded labels

    model = MLP(input_size, h1, h2, h3, output_size, dropout_rate=dropout_rate)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

    best_valid_loss = float('inf')
    patience_counter = 0
    train_errors = []

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(torch.tensor(X_train, dtype=torch.float))
        loss = criterion(outputs, torch.argmax(torch.tensor(y_train_onehot, dtype=torch.float), dim=1))
        loss.backward()
        optimizer.step()

        # Validation step
        model.eval()
        with torch.no_grad():
            valid_outputs = model(torch.tensor(X_valid, dtype=torch.float))
            valid_loss = criterion(valid_outputs, torch.argmax(torch.tensor(y_valid_onehot, dtype=torch.float), dim=1))

        # Early stopping
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_mlp_model.pth')
        else:
            patience_counter += 1

        train_errors.append(loss.item())
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}, Valid Loss: {valid_loss.item()}')

        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

        scheduler.step(valid_loss)

    # Load the best model
    model.load_state_dict(torch.load('best_mlp_model.pth'))

    # Testing the model and calculating confusion matrices
    with torch.no_grad():
        model.eval()
        train_outputs = model(torch.tensor(X_train, dtype=torch.float))
        valid_outputs = model(torch.tensor(X_valid, dtype=torch.float))
        test_outputs = model(torch.tensor(X_test, dtype=torch.float))

    # Get predicted labels
    train_preds = torch.argmax(train_outputs, dim=1).numpy()
    valid_preds = torch.argmax(valid_outputs, dim=1).numpy()
    test_preds = torch.argmax(test_outputs, dim=1).numpy()

    # Calculate confusion matrices
    train_cm = confusion_matrix(np.argmax(y_train_onehot, axis=1), train_preds)
    valid_cm = confusion_matrix(np.argmax(y_valid_onehot, axis=1), valid_preds)
    test_cm = confusion_matrix(np.argmax(y_test_onehot, axis=1), test_preds)

    # Calculate metrics
    train_metrics = calculate_metrics(train_cm)
    valid_metrics = calculate_metrics(valid_cm)
    test_metrics = calculate_metrics(test_cm)

    return train_cm, valid_cm, test_cm, train_errors, train_outputs, valid_outputs, test_outputs, \
           train_metrics, valid_metrics, test_metrics

# Main execution loop
num_runs = 10
train_cms, valid_cms, test_cms = [], [], []
train_errors_list, train_outputs_list, valid_outputs_list, test_outputs_list = [], [], [], []
train_metrics_list, valid_metrics_list, test_metrics_list = [], [], []

for run in range(num_runs):
    print(f'Run {run + 1}/{num_runs}')
    train_cm, valid_cm, test_cm, train_errors, train_outputs, valid_outputs, test_outputs, \
    train_metrics, valid_metrics, test_metrics = train_and_evaluate(
        X_train, y_train_onehot, X_valid, y_valid_onehot, X_test, y_test_onehot,
        num_epochs=200, lr=0.001, patience=10, weight_decay=1e-4, dropout_rate=0.3
    )
    train_cms.append(train_cm)
    valid_cms.append(valid_cm)
    test_cms.append(test_cm)
    train_errors_list.append(train_errors)
    train_outputs_list.append(train_outputs)
    valid_outputs_list.append(valid_outputs)
    test_outputs_list.append(test_outputs)
    train_metrics_list.append(train_metrics)
    valid_metrics_list.append(valid_metrics)
    test_metrics_list.append(test_metrics)

# Find the run with the lowest final validation error
best_run_idx = np.argmin([errors[-1] for errors in train_errors_list])
best_train_errors = train_errors_list[best_run_idx]
best_train_outputs = train_outputs_list[best_run_idx]
best_valid_outputs = valid_outputs_list[best_run_idx]
best_test_outputs = test_outputs_list[best_run_idx]

# Plot confusion matrices, ROC curves, and display metrics (as in your original code)
# Plotting functions
def plot_confusion_matrix(cm_mean, cm_std, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))  # Create a new figure for each confusion matrix
    plt.imshow(cm_mean, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(cm_mean))
    plt.xticks(tick_marks, tick_marks, rotation=45)
    plt.yticks(tick_marks, tick_marks)
    for i in range(len(cm_mean)):
        for j in range(len(cm_mean)):
            plt.text(j, i, f'{int(cm_mean[i, j])}',
                     horizontalalignment="center",
                     color="white" if cm_mean[i, j] > cm_mean.max() / 2 else "black")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Calculate average and standard deviation of confusion matrices
def calculate_cm_stats(cms):
    cms = np.array(cms)
    cm_mean = np.mean(cms, axis=0)
    cm_std = np.std(cms, axis=0)
    return cm_mean, cm_std

train_cm_mean, train_cm_std = calculate_cm_stats(train_cms)
valid_cm_mean, valid_cm_std = calculate_cm_stats(valid_cms)
test_cm_mean, test_cm_std = calculate_cm_stats(test_cms)

# Save Confusion Matrices
plot_confusion_matrix(train_cm_mean, train_cm_std, title='Confusion Matrix (Train)')
plt.savefig('/content/MLP_train_conf_matrix.svg', format='svg')
plt.close()

plot_confusion_matrix(valid_cm_mean, valid_cm_std, title='Confusion Matrix (Valid)')
plt.savefig('/content/MLP_valid_conf_matrix.svg', format='svg')
plt.close()

plot_confusion_matrix(test_cm_mean, test_cm_std, title='Confusion Matrix (Test)')
plt.savefig('/content/MLP_test_conf_matrix.svg', format='svg')
plt.close()

# Save ROC Curves
def plot_roc_curve(y_true, y_pred, title='ROC Curve'):
    plt.figure(figsize=(10, 8))  # Create a new figure for each ROC curve
    output_size = y_true.shape[1]
    for i in range(output_size):
        fpr, tpr, _ = roc_curve(y_true[:, i], y_pred.numpy()[:, i])
        auc = roc_auc_score(y_true[:, i], y_pred.numpy()[:, i])
        plt.plot(fpr, tpr, label=f'Class {i + 1} (AUC = {auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')  # Plot diagonal line
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend()

plot_roc_curve(y_train_onehot, best_train_outputs, title='ROC Curve (Train)')
plt.savefig('/content/MLP_train_roc.svg', format='svg')
plt.close()

plot_roc_curve(y_valid_onehot, best_valid_outputs, title='ROC Curve (Valid)')
plt.savefig('/content/MLP_valid_roc.svg', format='svg')
plt.close()

plot_roc_curve(y_test_onehot, best_test_outputs, title='ROC Curve (Test)')
plt.savefig('/content/MLP_test_roc.svg', format='svg')
plt.close()

# Extract and display the best metrics
best_train_metrics = train_metrics_list[best_run_idx]
best_valid_metrics = valid_metrics_list[best_run_idx]
best_test_metrics = test_metrics_list[best_run_idx]
print(f'Best Train Metrics (TP, TN, FP, FN, Sensitivity, Specificity, MCC): {best_train_metrics}')
print(f'Best Valid Metrics (TP, TN, FP, FN, Sensitivity, Specificity, MCC): {best_valid_metrics}')
print(f'Best Test Metrics (TP, TN, FP, FN, Sensitivity, Specificity, MCC): {best_test_metrics}')


## SVM model




In [ ]:
# Number of runs
n_runs = 10

# Initialize storage for results
conf_matrices_train = []
conf_matrices_valid = []
conf_matrices_test = []
roc_curves_train = []
roc_curves_valid = []
roc_curves_test = []
auc_train = []
auc_valid = []
auc_test = []

# Metrics storage
tp_train, tn_train, fp_train, fn_train = [], [], [], []
tp_valid, tn_valid, fp_valid, fn_valid = [], [], [], []
tp_test, tn_test, fp_test, fn_test = [], [], [], []
sensitivity_train, specificity_train, mcc_train = [], [], []
sensitivity_valid, specificity_valid, mcc_valid = [], [], []
sensitivity_test, specificity_test, mcc_test = [], [], []

for run in range(n_runs):
    # Initialize SVM classifier
    svm_classifier = SVC(kernel='rbf', C=0.1, gamma='scale', probability=True)

    # Train the SVM classifier
    svm_classifier.fit(X_train, y_train)

    # Save the model to a file
    joblib.dump(svm_classifier, 'svm_classifier.joblib')

    # Predict on the validation set
    y_pred_train = svm_classifier.predict(X_train)
    y_pred_valid = svm_classifier.predict(X_valid)
    y_pred_test = svm_classifier.predict(X_test)

    # Evaluate the model
    conf_matrix_train = confusion_matrix(y_train, y_pred_train)
    conf_matrix_valid = confusion_matrix(y_valid, y_pred_valid)
    conf_matrix_test = confusion_matrix(y_test, y_pred_test)

    conf_matrices_train.append(conf_matrix_train)
    conf_matrices_valid.append(conf_matrix_valid)
    conf_matrices_test.append(conf_matrix_test)

    # Calculate TP, TN, FP, FN, Sensitivity, Specificity, and MCC for train set
    tp = np.diag(conf_matrix_train)
    fp = conf_matrix_train.sum(axis=0) - tp
    fn = conf_matrix_train.sum(axis=1) - tp
    tn = conf_matrix_train.sum() - (fp + fn + tp)
    tp_train.append(tp.sum())
    tn_train.append(tn.sum())
    fp_train.append(fp.sum())
    fn_train.append(fn.sum())
    sensitivity_train.append(tp.sum() / (tp.sum() + fn.sum()))
    specificity_train.append(tn.sum() / (tn.sum() + fp.sum()))
    mcc_train.append(matthews_corrcoef(y_train, y_pred_train))

    # Calculate TP, TN, FP, FN, Sensitivity, Specificity, and MCC for valid set
    tp = np.diag(conf_matrix_valid)
    fp = conf_matrix_valid.sum(axis=0) - tp
    fn = conf_matrix_valid.sum(axis=1) - tp
    tn = conf_matrix_valid.sum() - (fp + fn + tp)
    tp_valid.append(tp.sum())
    tn_valid.append(tn.sum())
    fp_valid.append(fp.sum())
    fn_valid.append(fn.sum())
    sensitivity_valid.append(tp.sum() / (tp.sum() + fn.sum()))
    specificity_valid.append(tn.sum() / (tn.sum() + fp.sum()))
    mcc_valid.append(matthews_corrcoef(y_valid, y_pred_valid))

    # Calculate TP, TN, FP, FN, Sensitivity, Specificity, and MCC for test set
    tp = np.diag(conf_matrix_test)
    fp = conf_matrix_test.sum(axis=0) - tp
    fn = conf_matrix_test.sum(axis=1) - tp
    tn = conf_matrix_test.sum() - (fp + fn + tp)
    tp_test.append(tp.sum())
    tn_test.append(tn.sum())
    fp_test.append(fp.sum())
    fn_test.append(fn.sum())
    sensitivity_test.append(tp.sum() / (tp.sum() + fn.sum()))
    specificity_test.append(tn.sum() / (tn.sum() + fp.sum()))
    mcc_test.append(matthews_corrcoef(y_test, y_pred_test))

    # Obtain predicted probabilities
    train_probs = svm_classifier.decision_function(X_train)
    valid_probs = svm_classifier.decision_function(X_valid)
    test_probs = svm_classifier.decision_function(X_test)

    # ROC curves for each class
    n_classes = len(np.unique(y_train))
    roc_curves_train.append([roc_curve((y_train == i).astype(int), train_probs[:, i]) for i in range(n_classes)])
    roc_curves_valid.append([roc_curve((y_valid == i).astype(int), valid_probs[:, i]) for i in range(n_classes)])
    roc_curves_test.append([roc_curve((y_test == i).astype(int), test_probs[:, i]) for i in range(n_classes)])

    # AUC for each class
    auc_train.append([roc_auc_score((y_train == i).astype(int), train_probs[:, i]) for i in range(n_classes)])
    auc_valid.append([roc_auc_score((y_valid == i).astype(int), valid_probs[:, i]) for i in range(n_classes)])
    auc_test.append([roc_auc_score((y_test == i).astype(int), test_probs[:, i]) for i in range(n_classes)])

# Function to plot confusion matrix
def plot_conf_matrix(conf_matrices, filename, title):
    plt.figure(figsize=(8, 6))  # Match the figure size used in the MLP code

    # Calculate the mean and standard deviation of the confusion matrices
    mean_conf_matrix = np.mean(conf_matrices, axis=0)
    std_conf_matrix = np.std(conf_matrices, axis=0)

    # Plotting the mean confusion matrix
    plt.imshow(mean_conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title, fontsize=14)  # Match the font size used in the MLP code
    plt.colorbar()

    tick_marks = np.arange(len(mean_conf_matrix))
    plt.xticks(tick_marks, tick_marks, rotation=45, fontsize=12)  # Match the font size for ticks
    plt.yticks(tick_marks, tick_marks, fontsize=12)  # Match the font size for ticks

    # Adding text annotations for each cell
    for i in range(len(mean_conf_matrix)):
        for j in range(len(mean_conf_matrix)):
            plt.text(
                j, i,
                f'{int(mean_conf_matrix[i, j])}',
                horizontalalignment="center",
                color="white" if mean_conf_matrix[i, j] > mean_conf_matrix.max() / 2 else "black",
                fontsize=12  # Match the font size for annotations
            )

    plt.ylabel('True Label', fontsize=14)  # Match the font size for y-label
    plt.xlabel('Predicted Label', fontsize=14)  # Match the font size for x-label
    plt.tight_layout()  # Ensure proper spacing
    plt.savefig(filename, format='svg')
    plt.close()


# Function to plot ROC curve
def plot_roc_curve(roc_curve_data, auc_data, filename):  # , title):
    plt.figure(figsize=(10, 8))  # Match the figure size used in the MLP code

    for class_idx in range(len(roc_curve_data[0])):
        mean_fpr = np.mean([curve[class_idx][0] for curve in roc_curve_data], axis=0)
        mean_tpr = np.mean([curve[class_idx][1] for curve in roc_curve_data], axis=0)
        mean_auc = np.mean([auc[class_idx] for auc in auc_data])
        plt.plot(mean_fpr, mean_tpr, label=f'Class {class_idx + 1} (AUC = {mean_auc:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line
    plt.xlabel('False Positive Rate', fontsize=14)  # Match the font size for x-label
    plt.ylabel('True Positive Rate', fontsize=14)  # Match the font size for y-label
    plt.legend(loc='best', fontsize=12)  # Match the font size for the legend
    plt.xticks(fontsize=12)  # Match the font size for ticks
    plt.yticks(fontsize=12)  # Match the font size for ticks
    plt.tight_layout()  # Ensure proper spacing
    plt.savefig(filename, format='svg')
    plt.close()


# Plot and save confusion matrices
plot_conf_matrix(conf_matrices_train, '/content/SVM_train_conf_matrix.svg', 'Confusion Matrix (Train)')
plot_conf_matrix(conf_matrices_valid, '/content/SVM_valid_conf_matrix.svg', 'Confusion Matrix (Valid)')
plot_conf_matrix(conf_matrices_test, '/content/SVM_test_conf_matrix.svg', 'Confusion Matrix (Test)')

# Plot and save ROC curves
plot_roc_curve(roc_curves_train, auc_train, '/content/SVM_train_roc_curve.svg')  # , 'Train ROC Curve')
plot_roc_curve(roc_curves_valid, auc_valid, '/content/SVM_valid_roc_curve.svg')  # , 'Valid ROC Curve')
plot_roc_curve(roc_curves_test, auc_test, '/content/SVM_test_roc_curve.svg')  # , 'Test ROC Curve')

# Print metrics
print(f"Train - TP: {np.mean(tp_train):.2f}, TN: {np.mean(tn_train):.2f}, FP: {np.mean(fp_train):.2f}, FN: {np.mean(fn_train):.2f}")
print(f"Train - Sensitivity: {np.mean(sensitivity_train):.2f}, Specificity: {np.mean(specificity_train):.2f}, MCC: {np.mean(mcc_train):.2f}")
print(f"Valid - TP: {np.mean(tp_valid):.2f}, TN: {np.mean(tn_valid):.2f}, FP: {np.mean(fp_valid):.2f}, FN: {np.mean(fn_valid):.2f}")
print(f"Valid - Sensitivity: {np.mean(sensitivity_valid):.2f}, Specificity: {np.mean(specificity_valid):.2f}, MCC: {np.mean(mcc_valid):.2f}")
print(f"Test - TP: {np.mean(tp_test):.2f}, TN: {np.mean(tn_test):.2f}, FP: {np.mean(fp_test):.2f}, FN: {np.mean(fn_test):.2f}")
print(f"Test - Sensitivity: {np.mean(sensitivity_test):.2f}, Specificity: {np.mean(specificity_test):.2f}, MCC: {np.mean(mcc_test):.2f}")